In [ ]:
import torch
import matplotlib.pyplot as plt
import numpy as np
from torch.utils.data import DataLoader
from torchvision import transforms 
from torch import optim

%load_ext autoreload
%autoreload 3

from model import CarlaNet
from utils import visualizeImages , cost ,loss_mask  , StopEarly 
from data import AgentData , ToTensor 


### Set up training environment 

In [ ]:
batch_size = 200
BRANCH_LOSS_WEIGHT= [0.95, 0.95, 0.95, 0.95] # how much each branch is weighted when computing loss
# params used in loss, we will update this None values while training 
params = {"branch_weights":BRANCH_LOSS_WEIGHT,\
          # how much each of the outputs specified on TARGETS are weighted for learning.
         "variable_weights":{"Steer":0.5,"Gas":0.45,"Brake":0.05},\
          # values we used them while training 
         "branches":None , "targets": None , "controls_mask":None}
loss_type = "L1"
model = CarlaNet().cuda()
opt = optim.SGD(model.parameters(),lr=0.01,momentum=0.9,nesterov=True)

## prepare Data

In [ ]:
transform = transforms.Compose([ToTensor()])
training_data = AgentData('./SeqTrain',transforms=transform)
train_loader = DataLoader(training_data,batch_size=batch_size)
data_iterator = iter(train_loader)

In [ ]:
# info about Data 
print(f"num of trainin samples is : {len(training_data)}")

In [ ]:
num_of_batches = len(training_data)/batch_size
num_of_batches

## Training

In [ ]:
def train_net(n_epochs):

    # prepare the net for training
    model.train()

    for epoch in range(n_epochs):  # loop over the dataset multiple times
        
        running_loss = 0.0
        epoch_loss = 0.0
        
        # train on batches of data, assumes you already have train_loader
        for batch_i, data in enumerate(train_loader):
            # get inputs(img,speed,command) and targets(steer,gas,brake)
            inputs ,targets = data
            imgs , speed , command = inputs
            imgs , speed , command = imgs.type(torch.cuda.FloatTensor) ,speed.type(torch.cuda.FloatTensor)\
                                    ,command.type(torch.cuda.LongTensor)
            
            speed , command = speed.view((batch_size,1)) , command.view((batch_size,1))
            inputs = (imgs , speed , command)
            
            # targets 
            steer , gas , brake = targets
            targets = torch.stack([steer,gas,brake],dim=1).cuda()

            # forward pass to get outputs
            outs = model(inputs)
            controls_masks = loss_mask(command)
            
            #update params
            params["branches"] = outs
            params["targets"] = targets
            params["controls_mask"] = controls_masks

            # calculate the loss between predicted and target keypoints
            loss = cost(params=params,type_loss=loss_type)

            # zero the parameter (weight) gradients
            opt.zero_grad()
            
            # backward pass to calculate the weight gradients
            loss.backward()

            # update the weights
            opt.step()

            # print loss statistics
            # to convert loss into a scalar and add it to the running_loss, use .item()
            running_loss += loss.item()
            epoch_loss += running_loss
            if batch_i +1 % 100 == 0:    # print every 100 batches
                print('Epoch: {}, Batch: {}, Avg. Loss: {}'.format(epoch + 1, batch_i+1, running_loss/100))
                running_loss = 0.0
        print('----------------\nEpoch loss: {}\n------------\n'.format(epoch_loss/num_of_batches))
        if stop_early(model,epoch_loss/num_of_batches) :
            print("Early stop activated ... stoping training process..")
            break
    print('Finished Training')

In [ ]:
torch.cuda.empty_cache() # empty GPU cash 

In [ ]:
train_net(1) # test train with 1 iter